### Importação das Libs

In [1]:
import pandas as pd
import numpy as np
import pickle
import json

#Pacotes Estatísticos
import scipy
from scipy import stats

#ML
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

### Variáveis de Configuração

In [2]:
local_origem = "C:/Lab Transp/Dados Processados/"
local_destino = "C:/Lab Transp/Dados Processados/"

data_inicio = "2014-01-01"
data_fim = ""

In [3]:
class Manipulacao_Arquivos:
        
    def abrir_arquivo(self, caminho, arquivo):
        try:
            df = pd.read_csv(caminho + "\\" + arquivo + ".csv", encoding="latin-1", decimal=".",  low_memory=False)     
            return df
        except:
            try:
                df = pd.read_csv(caminho + "\\" + arquivo + ".csv", encoding="latin-1", sep=";", decimal=".",  low_memory=False)
                return df
            except Exception as error:
                print('Erro ao carregar arquivo: ' + repr(error))
                return None
    
    
    def criar_arquivo_primario(self, caminho, arquivo_blihetagem = "BD_Bilhetagem", arquivo_linhas = "BD_Linhas_Geoespacial"):
        #Abro os arquivos de origem
        df_bilhetagem = self.abrir_arquivo(local_origem, "BD_Bilhetagem")
        df_linhas = self.abrir_arquivo(local_origem, "BD_Linhas_Geoespacial")
        
        df_bilhetagem['LINHA_TRATADA'] = df_bilhetagem['LINHA'].str.split('-').str[0]
        df_bilhetagem['LINHA_TRATADA'] = df_bilhetagem['LINHA_TRATADA'].astype(str)
        df_bilhetagem['LINHA_TRATADA'] = df_bilhetagem['LINHA_TRATADA'].apply(lambda x: x.strip())        

        df_final = df_bilhetagem.merge(df_linhas[['ln_nome_tp', 
                                                  'ln_empresa_tp',
                                                  'ln_codigo_tp',
                                                  'pt_nome_tp',
                                                  'ds_subpref_tp', 
                                                  'latitude_longitude_tp',           
                                                  'pt_nome_ts',
                                                  'ds_subpref_ts', 
                                                  'latitude_longitude_ts']],
                                           left_on="LINHA_TRATADA",
                                           right_on="ln_codigo_tp",
                                           how="left")
        
        #Tratar colunas de texto
        df_final = self.tratar_colunas_strings(df_final, ['EMPRESA'])        
        
        #Tratar colunas de data
        df_final = self.tratar_colunas_datas(df_final, ['DATA'])
        
        df_final["ANO"] = df_final["DATA"].dt.year
        df_final["ANO"] = df_final["ANO"].fillna(0).astype(int)        
        
        df_final["MES"] = df_final["DATA"].dt.month
        df_final["MES"] = df_final["MES"].fillna(0).astype(int)
        
        df_final["DIA"] = df_final["DATA"].dt.day
        df_final["DIA"] = df_final["DIA"].fillna(0).astype(int)
        
        
        #Filtrar pelo data de corte
        df_final = df_final[df_final['DATA']>=data_inicio]
        
        
        #Tratar colunas numéricas
        df_final = self.tratar_colunas_numericas(df_final, ['PASSAGEIROS_GRATUIDADE', 'PASSAGEIROS_GRATUIDADE_ESTUDANTE', 'PASSAGEIROS_PAGANTES',
                                                            'PASSAGEIROS_PAGANTES_BU_COMUM', 'PASSAGEIROS_PAGANTES_BU_COMUM_MENSAL', 'PASSAGEIROS_PAGANTES_BU_ESTUDANTES',
                                                            'PASSAGEIROS_PAGANTES_BU_ESTUDANTES_MENSAL', 'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP', 'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP_MENSAL',
                                                            'PASSAGEIROS_PAGANTES_DINHEIRO', 'PASSAGEIROS_PAGANTES_ESTUDANTES', 'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_METRO_CPTM',
                                                            'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_ONIBUS', 'TOTAL'])
        
        
        #Consolidação dos Tipos de Passageiros
        df_final['VOLUME_TOTAL_PASSAGEIROS_PAGANTES'] = df_final['PASSAGEIROS_PAGANTES']        
        df_final['PAGANTES_DINHEIRO'] = df_final['PASSAGEIROS_PAGANTES_DINHEIRO']
        df_final['VOLUME_TOTAL_PASSAGEIROS_GRATUIDADE'] = df_final['PASSAGEIROS_GRATUIDADE'] + df_final['PASSAGEIROS_GRATUIDADE_ESTUDANTE']
        df_final['BU_COMUM'] = df_final['PASSAGEIROS_PAGANTES_BU_COMUM'] + df_final['PASSAGEIROS_PAGANTES_BU_COMUM_MENSAL']
        df_final['BU_VALE_TRANSPORTE'] = df_final['PASSAGEIROS_PAGANTES_BU_VALE_TRANSP'] + df_final['PASSAGEIROS_PAGANTES_BU_VALE_TRANSP_MENSAL']
        df_final['INTEGRACAO'] = df_final['PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_METRO_CPTM'] + df_final['PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_ONIBUS']
        df_final['BU_ESTUDANTE'] = df_final['PASSAGEIROS_PAGANTES_BU_ESTUDANTES'] + df_final['PASSAGEIROS_PAGANTES_BU_ESTUDANTES_MENSAL'] + df_final['PASSAGEIROS_PAGANTES_ESTUDANTES']
        
        #Para fins deste estudo, estamos usando essas variáveis
        df_final['VOLUME_TOTAL_PASSAGEIROS_BU_VT'] = df_final['BU_COMUM'] + df_final['BU_VALE_TRANSPORTE']        
        df_final['VOLUME_TOTAL_PASSAGEIROS_BU_VT_DIN'] = df_final['VOLUME_TOTAL_PASSAGEIROS_BU_VT'] + df_final['PAGANTES_DINHEIRO']
        
        
        #Depois das todas as somas, eu volto os zeros para NAN para tratá-los mais facilmente depois
        df_final['VOLUME_TOTAL_PASSAGEIROS_BU_VT'] = df_final['VOLUME_TOTAL_PASSAGEIROS_BU_VT'].replace(0, np.nan)
        df_final['VOLUME_TOTAL_PASSAGEIROS_BU_VT_DIN'] = df_final['VOLUME_TOTAL_PASSAGEIROS_BU_VT_DIN'].replace(0, np.nan)
        
        '''
        A especificação é que o volume de interesse fosse BU + VT.
        O Problema é que nem sempre essas colunas retornam preechidas corretamente
        A sugestão é utilizar a 'PASSAGEIROS_PAGANTES'
        '''
        #df_final['VOLUME_TOTAL_PASSAGEIROS_INTERESSE'] = df_final['BU_COMUM'] + df_final['BU_VALE_TRANSPORTE'] 

        '''
        Em alguns casos, apesar da linha ter o mesmo nome completo, não a encontro via o nome tratado. 
        Assim, crio um dicionário por Linha_Completa e a Sub_Prefeitura mapeada, excluindo os que ficaram ausentes
        Então, remapeio a coluna Sub_Prefeitura com os valores preenchidos
        '''        
        
        #Crio o dicionário
        ref_sub = df_final[['LINHA', 'ds_subpref_tp']].drop_duplicates()
        ref_sub = ref_sub[np.logical_not(ref_sub['ds_subpref_tp'].isna())]
        ref_sub_dict = dict(zip(ref_sub.LINHA, ref_sub.ds_subpref_tp))

        #Substituo os valores
        df_final['ds_subpref_tp'] = df_final['LINHA'].map(ref_sub_dict)
        
        ref_sub= None
        ref_sub_dict= None
        
            
        #Carrego a Zona da Sub-Prefeitura do Ponto Inicial
        with open(local_origem + "\zonas_sub_pref.json") as dic_zonas:
            dict_zonas_sub_pref = json.load(dic_zonas)        
            
        df_final['Zona'] = df_final['ds_subpref_tp'].map(dict_zonas_sub_pref)
        
        
        #Renomeio as colunas
        df_final.rename(columns={'NOME_ARQUIVO' : 'Nome_Arquivo', 
                                 'DATA' : 'Data', 
                                 'AREA' : 'Area', 
                                 'EMPRESA' : 'Empresa', 
                                 'LINHA' : 'Linha_Completa', 
                                 'TIPO' : 'Tipo', 
                                 'TOTAL' : 'Total', 
                                 'ANO' : 'Ano', 
                                 'MES' : 'Mes', 
                                 'DIA' : 'Dia', 
                                 'LINHA_TRATADA' : 'Linha',                                 
                                 'ds_subpref_tp' : 'Sub_Prefeitura',
                                 'pt_nome_tp'    : 'Nome_Ponto_Inicial',
                                 'pt_nome_ts'    : 'Nome_Ponto_Final',
                                 'latitude_longitude_tp' : 'Lat_Long_Ponto_Inicial',
                                 'latitude_longitude_ts' : 'Lat_Long_Ponto_Final',
                                 'VOLUME_TOTAL_PASSAGEIROS_PAGANTES' : 'Volume_Passageiros_Pagantes', 
                                 'PAGANTES_DINHEIRO' : 'Volume_Pagantes_Dinheiro', 
                                 'VOLUME_TOTAL_PASSAGEIROS_GRATUIDADE' : 'Volume_Passageiros_Gratuidade', 
                                 'BU_COMUM' : 'Volume_BU_Comum', 
                                 'BU_VALE_TRANSPORTE' : 'Volume_BU_Vale_Transporte', 
                                 'INTEGRACAO' : 'Volume_Integracao', 
                                 'BU_ESTUDANTE' : 'Volume_BU_Estudante', 
                                 'VOLUME_TOTAL_PASSAGEIROS_BU_VT' : 'Volume_Passageiros_BU_VT', 
                                 'VOLUME_TOTAL_PASSAGEIROS_BU_VT_DIN' : 'Volume_Passageiros_BU_VT_DIN'}, inplace=True)          

        
        perc_encontrados = len(np.unique(df_final[df_final['ln_codigo_tp'].notnull()]['ln_nome_tp']))/len(np.unique(df_final['Linha']))
        print(f'{perc_encontrados: .1%} das linhas em Bilhetagem encontrados no cadastro georeferencial de linhas')
    
        #Depois de consolidar os valores, dropo as colunas        
        df_final.drop(columns=['ds_subpref_ts', 'ln_nome_tp', 'ln_empresa_tp', 'ln_codigo_tp', 'PASSAGEIROS_GRATUIDADE', 'PASSAGEIROS_GRATUIDADE_ESTUDANTE', 'PASSAGEIROS_PAGANTES', 'PASSAGEIROS_PAGANTES_BU_COMUM',
                               'PASSAGEIROS_PAGANTES_BU_COMUM_MENSAL', 'PASSAGEIROS_PAGANTES_BU_ESTUDANTES', 'PASSAGEIROS_PAGANTES_BU_ESTUDANTES_MENSAL',
                               'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP', 'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP_MENSAL', 'PASSAGEIROS_PAGANTES_DINHEIRO', 'PASSAGEIROS_PAGANTES_ESTUDANTES',
                               'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_METRO_CPTM', 'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_ONIBUS'], inplace=True)
                    
        return df_final
    
    def tratar_colunas_strings(self, df, colunas):
        
        for col in colunas:
            df[col] = df[col].astype(str)
            df[col] = df[col].apply(lambda x: x.strip().upper())
            
        return df
    
    def tratar_colunas_datas(self, df, colunas):
        for col in colunas:
            df[col] = df[col].astype(str)
            df[col] = df[col].apply(lambda x: x.strip(" 00:00:00"))
            df[col] = pd.to_datetime(df[col])
        
        return df
    
    def tratar_colunas_numericas(self, df, colunas):
        for col in colunas:
            df[col] = df[col].astype(float)
            df[col] = df[col].fillna(0)            
        return df
                
    def salvar_arquivo(self, caminho, df, arquivo):
        df.to_csv(caminho + "\\" + arquivo + ".csv" , index=False, encoding="latin-1", decimal=",")        
        return None
    
    def lista_agregadora(self, df, coluna="Sub_Prefeitura"):
        df[coluna] = df[coluna].astype(str)
        return np.unique(df[coluna].to_numpy())
    
    
    def selecao_linhas(self, df, apenas_nao_terminais, limiar_dias, manter_apenas_modalidades_pagamentos_primarias):
        #Manteho apenas as linhas que não terminam em terminais
        if apenas_nao_terminais==True:
            df = df[np.logical_not(df['Nome_Ponto_Inicial'].str.contains('TERM'))].reset_index(drop=True)
            df = df[np.logical_not(df['Nome_Ponto_Final'].str.contains('TERM'))].reset_index(drop=True)

        #Removo as linhas com registros abaixo do limiar de dias definido
        g_amostra = df[['Linha']].groupby('Linha').size().reset_index()
        linhas_selecionadas = g_amostra[g_amostra[0]>=limiar_dias]['Linha']
        df[df['Linha'].isin(linhas_selecionadas)].reset_index(drop=True)   


        if manter_apenas_modalidades_pagamentos_primarias==True:
            df.dropna(subset=['Volume_Passageiros_BU_VT'], inplace=True)
        
        return df
            
    def definicao_linhas_maes(self, df):
        df_linhas_sub_pref = df[['Sub_Prefeitura', 'Zona', 'Data', 'Linha', 'Nome_Ponto_Inicial', 'Nome_Ponto_Final']].groupby(['Zona', 'Sub_Prefeitura', 'Linha',  'Nome_Ponto_Inicial', 'Nome_Ponto_Final'], as_index=False).count().sort_values(by=['Zona', 'Sub_Prefeitura','Data'], ascending=True)

        df_linha_mae = df_linhas_sub_pref.groupby(by=['Zona', 'Sub_Prefeitura'], 
                                                  as_index=False, 
                                                  sort=False).apply(lambda x: x.loc[x["Data"].idxmax()])
        
        return df_linhas_sub_pref, df_linha_mae

In [4]:
class PreProcessamento:
    
    def tratar_dados_ausentes_(self, df, coluna="Volume_Passageiros_BU_VT_DIN", grupo="Linha", metodo="remover"):

        if metodo.lower()=="remover":
            df = df.dropna(subset=[coluna])
            
        elif metodo.lower()=="zero":
            df[coluna] = df[coluna].fillna(0)
                
        elif metodo.lower()=="média":
            df[coluna] = df[coluna].fillna(df.groupby(grupo)[coluna].transform('mean'))
            
        elif metodo.lower()=="mediana":
            df[coluna] = df[coluna].fillna(df.groupby(grupo)[coluna].transform('median'))
            
        elif metodo.lower()=="interpolação":
            #Quanto interpolamos, algumas colunas continuam com NaN. 
            #O ideal será chamar a função 'tratar_dados_ausentes' com um novo método            
            df = df.groupby('Linha', as_index=False).apply(lambda group: group.interpolate(method='linear')).reset_index(drop=True)
            
        else:
            print("ERRO: Método de tratamento não identificado...")
            print("Especifique um metódo dentre as opções: Remover, Zero, Média, Mediana ou Interpolação")
            print("O dataframe não foi alterado")
        
        return df
    
    def corrigir_outliers(self, df, coluna="Volume_Passageiros_BU_VT_DIN", metodo="interquartil"):
                
        if metodo.lower()=="interquartil":
            for i, linha in enumerate(np.unique(df['Linha'])):
                
                #Definição dos Quartis
                Q1 = df_final[df_final['Linha']==linha]['Volume_Passageiros_BU_VT'].quantile(0.25)
                Q3 = df_final[df_final['Linha']==linha]['Volume_Passageiros_BU_VT'].quantile(0.75)
                IQR = Q3 - Q1

                #Diferença Interquartil
                limite_inferior = Q1 - 1.5*IQR
                limite_superior = Q3 + 1.5*IQR    

                #Filtro dos valores limites
                fora_limite_inferior = np.where((df_final['Linha']==linha) & (df_final['Volume_Passageiros_BU_VT'] <= limite_inferior))[0]
                fora_limite_superior = np.where((df_final['Linha']==linha) & (df_final['Volume_Passageiros_BU_VT'] >= limite_superior))[0]

                #Remoção dos Outliers
                if i == 0:
                    df_sem_outliers = df_final.drop(index=fora_limite_superior)
                    df_sem_outliers = df_final.drop(index=fora_limite_inferior)
                    df_sem_outliers.reset_index(drop=True, inplace=True)
                else:
                    df_sem_outliers = df_sem_outliers.drop(index=fora_limite_superior)
                    df_sem_outliers = df_sem_outliers.drop(index=fora_limite_inferior)
                    df_sem_outliers.reset_index(drop=True, inplace=True)
                    
        
        elif metodo.lower()=="zscore":
            z = np.abs(stats.zscore(df[coluna]))
            
            limite_z = 2
            outlier_indices = np.where(z > limite_z)[0]
            df_sem_outliers = df.drop(index=outlier_indices)
            
        else:
            print("ERRO: Método de transformação não identificado...")
            print("Especifique um metódo dentre as opções: Log, Diferenciação, Pontencia, Raiz Qaudrada ou Box-Cox")
            print("O dataframe não foi alterado")
            
        
        return df_sem_outliers
    
    def transformacao_dados_(self, df, coluna="Volume_Passageiros_BU_VT_DIN", grupo="LINHA_TRATADA", metodo="nenhum"):
       
        if metodo.lower()=="log":
            df[coluna] = np.log(df[coluna])
            
        elif metodo.lower()=="diferenciacao":            
            df[coluna+"_diff"] = df.groupby([grupo])[coluna].diff().fillna(0)
            
        elif metodo.lower()=="potencia":
            df[coluna] = np.power(df[coluna], 2)
            
        elif metodo.lower()=="raiz quadrada":
            df[coluna] = np.sqrt(df[coluna])
                        
        elif metodo.lower()=="box cox":
            df[coluna] = scipy.stats.boxcox(df[coluna])[0]
        
        elif metodo.lower()=="nenhum":
            print("O dataframe não foi alterado")
            
        else:
            print("ERRO: Método de transformação não identificado...")
            print("Especifique um metódo dentre as opções: Log, Diferenciação, Pontencia, Raiz Qaudrada ou Box-Cox")
            print("O dataframe não foi alterado")
            
        return df
    
    def normalizacao_dados_(self, df, coluna="VOLUME_TOTAL_PASSAGEIROS", metodo="Standard Scaler"):
         
        if metodo.lower()=="standard scaler":                
            scaler = StandardScaler()
            scaler.fit(df[coluna].to_numpy().reshape(-1, 1))

        elif metodo.lower()=="mix max scaler":
            scaler = MinMaxScaler()
            scaler.fit(df[coluna].to_numpy().reshape(-1, 1))

        else:
            print("ERRO: Método de normalização não identificado...")
            print("Especifique um metódo dentre as opções: Standard Scaler ou MinMax Scaler")
            print("O dataframe não foi alterado")

        return df
    
    def selecao_linhas_(self, df):
        return df

## PRÉ-PROCESSAMENTO

### 1. Abrir arquivos de linhas

In [5]:
Manipulacao = Manipulacao_Arquivos()

In [6]:
df_final = Manipulacao.criar_arquivo_primario(local_origem,
                                              arquivo_blihetagem = "BD_Bilhetagem", 
                                              arquivo_linhas = "BD_Linhas_Geoespacial")

Erro ao carregar arquivo: FileNotFoundError(2, 'No such file or directory')


TypeError: 'NoneType' object is not subscriptable

In [10]:
df_final.tail(10)

,Nome_Arquivo,Data,Area,Empresa,Linha_Completa,Tipo,Total,Linha,Nome_Ponto_Inicial,Sub_Prefeitura,...,Volume_Passageiros_Pagantes,Volume_Pagantes_Dinheiro,Volume_Passageiros_Gratuidade,Volume_BU_Comum,Volume_BU_Vale_Transporte,Volume_Integracao,Volume_BU_Estudante,Volume_Passageiros_BU_VT,Volume_Passageiros_BU_VT_DIN,Zona
8518762,C:\Users\f112596\Desktop\Projetos\DO\Projeto V...,2024-04-09,NaN,GATUSA,709P10 - TERM GUIDO/PINHEIROS,NaN,5991.0,709P10,"EST. STO. AMARO , 0",PINHEIROS,...,3348.0,89.0,463.0,177.0,0.0,1827.0,87.0,177.0,266.0,OESTE
8518763,C:\Users\f112596\Desktop\Projetos\DO\Projeto V...,2024-04-09,NaN,GATUSA,709P10 - TERM GUIDO/PINHEIROS,NaN,5991.0,709P10,"TERM. PINHEIROS - PL 6 , 0",PINHEIROS,...,3348.0,89.0,463.0,177.0,0.0,1827.0,87.0,177.0,266.0,OESTE
8518764,C:\Users\f112596\Desktop\Projetos\DO\Projeto V...,2024-04-09,NaN,GATUSA,724510 - MORUMBI SHOPPING/EST LUZ,NaN,9225.0,724510,"TERM. STO. AMARO , 0",SE,...,5133.0,287.0,790.0,276.0,0.0,2399.0,340.0,276.0,563.0,CENTRO
8518765,C:\Users\f112596\Desktop\Projetos\DO\Projeto V...,2024-04-09,NaN,GATUSA,724510 - MORUMBI SHOPPING/EST LUZ,NaN,9225.0,724510,"R. MATO GROSSO , 388",SE,...,5133.0,287.0,790.0,276.0,0.0,2399.0,340.0,276.0,563.0,CENTRO
8518766,C:\Users\f112596\Desktop\Projetos\DO\Projeto V...,2024-04-09,NaN,GATUSA,755010 - TERM STO AMARO/M SANTA CECILIA,NaN,5456.0,755010,"TERM. STO. AMARO , 0",SE,...,2970.0,191.0,912.0,140.0,0.0,1092.0,151.0,140.0,331.0,CENTRO
8518767,C:\Users\f112596\Desktop\Projetos\DO\Projeto V...,2024-04-09,NaN,GATUSA,755010 - TERM STO AMARO/M SANTA CECILIA,NaN,5456.0,755010,"TERM. AMARAL GURGEL , 0",SE,...,2970.0,191.0,912.0,140.0,0.0,1092.0,151.0,140.0,331.0,CENTRO
8518768,C:\Users\f112596\Desktop\Projetos\DO\Projeto V...,2024-04-09,NaN,397,9927PR - TERM STO AMARO/TERM BANDEIRA,NaN,5282.0,9927PR,NaN,NaN,...,715.0,31.0,354.0,175.0,0.0,3997.0,10.0,175.0,206.0,NaN
8518769,C:\Users\f112596\Desktop\Projetos\DO\Projeto V...,2024-04-09,NaN,397,9928PR - TERM BANDEIRA/TERM STO AMARO,NaN,4620.0,9928PR,NaN,NaN,...,2132.0,62.0,350.0,200.0,0.0,1827.0,49.0,200.0,262.0,NaN
8518770,C:\Users\f112596\Desktop\Projetos\DO\Projeto V...,2024-04-09,NaN,292,9945PR - TERMINAL BUTANTA/GATO PRETO,NaN,504.0,9945PR,NaN,NaN,...,251.0,0.0,76.0,2.0,0.0,115.0,60.0,2.0,2.0,NaN
8518771,C:\Users\f112596\Desktop\Projetos\DO\Projeto V...,2024-04-09,NaN,901,SMATEU - TERM,NaN,11208.0,SMATEU,NaN,NaN,...,5360.0,0.0,1866.0,263.0,0.0,3456.0,263.0,263.0,263.0,NaN


### 2. Seleção de Linhas a serem tratadas

##### Critérios definidos para a seleção das linhas a serem tratadas
        i. Todos cujo destino não seja Terminais;
        ii. Quantos dias distintos eu tenho para cada linha? Enviar para o Roberto DF na segunda 26/02;
        iii. Validar existência de outras modalidades de pagamento, além de BU e VT

In [11]:
df_final = Manipulacao.selecao_linhas(df = df_final, 
                                      apenas_nao_terminais=True,
                                      limiar_dias=3650, 
                                      manter_apenas_modalidades_pagamentos_primarias=False)

In [12]:
df_final[['Ano', 'Volume_Passageiros_BU_VT_DIN']].groupby('Ano').sum('Volume_Passageiros_BU_VT_DIN')/1000

,Volume_Passageiros_BU_VT_DIN
Ano,
2014,1423874.171
2015,1366570.730
2016,1290559.035
2017,1258876.711
2018,1193380.469
2019,613280.496
2020,128954.686
2021,144698.771
2022,410180.824


#### 2.1 Definição da "Linha Mãe" de cada sub prefeitura?

In [13]:
df_linhas_sub_pref, df_linha_mae = Manipulacao.definicao_linhas_maes(df_final)

In [14]:
df_linhas_sub_pref.rename(columns={'ds_subpref_tp' : 'Sub_Prefeitura',
                                   'LINHA_TRATADA' : 'Linha',
                                   'pt_nome_tp'    : 'Nome_Ponto_Inicial',
                                   'pt_nome_ts'    : 'Nome_Ponto_Final',
                                   'DATA'          : 'Data'}, inplace=True)

df_linha_mae.rename(columns={'ds_subpref_tp' : 'Sub_Prefeitura',
                                   'LINHA_TRATADA' : 'Linha',
                                   'pt_nome_tp'    : 'Nome_Ponto_Inicial',
                                   'pt_nome_ts'    : 'Nome_Ponto_Final',
                                   'DATA'          : 'Data'}, inplace=True)

In [15]:
#Defino as linhas mães de cada subprefeitura
df_linha_mae = df_linhas_sub_pref.groupby(by=['Sub_Prefeitura'], 
                                          as_index=False, 
                                          sort=False).apply(lambda x: x.loc[x["Data"].idxmax()])

In [16]:
#Recupero as demais linhas de cada subprefeitura
df_linhas = df_linhas_sub_pref.groupby(by=['Zona', 'Sub_Prefeitura', 'Linha', 'Nome_Ponto_Inicial', 'Nome_Ponto_Final'], 
                                       as_index=False, 
                                       sort=False).sum('Data')
df_linhas.rename(columns={'Data' : 'Qtde_Dias'}, inplace=True)

In [17]:
#Criação do Boolean de indeitifcação (Mãe, Mantinda, Removida)
df_linhas['Mantido'] = np.where(df_linhas['Qtde_Dias']>=3650, 'Mantido', 'Removido')
df_linhas['Linha_Mae'] = np.where(df_linhas['Linha'].isin(df_linha_mae['Linha'].values), 'Mãe', 'Normal')

In [18]:
df_linhas[['Zona', 'Sub_Prefeitura', 'Mantido', 'Linha']].groupby(['Zona', 'Sub_Prefeitura', 'Mantido'], as_index=False).count().pivot(index=['Zona', 'Sub_Prefeitura'],
                                                                                                                       columns='Mantido',
                                                                                                                       values='Linha').fillna(0)

Mantido                           Mantido  Removido
Zona   Sub_Prefeitura                              
CENTRO SE                           110.0      50.0
LESTE  ARICANDUVA-FORMOSA-CARRAO     12.0       3.0
       ERMELINO MATARAZZO             1.0       0.0
       GUAIANASES                    17.0       3.0
       ITAIM PAULISTA                 6.0       4.0
       ITAQUERA                      52.0      13.0
       MOOCA                         60.0      13.0
       PENHA                         76.0      13.0
       SAO MATEUS                    24.0       5.0
       SAO MIGUEL                     8.0       1.0
       SAPOPEMBA                      3.0       1.0
       VILA PRUDENTE                  8.0       5.0
NORTE  CASA VERDE-CACHOEIRINHA       14.0       4.0
       FREGUESIA-BRASILANDIA          5.0       7.0
       JACANA-TREMEMBE                5.0       2.0
       PERUS                          1.0       3.0
       PIRITUBA-JARAGUA              11.0       8.0
       SANTANA-TUCURUVI              44.0      16.0
       VILA MARIA-VILA GUILHERME     18.0       6.0
OESTE  BUTANTA                       24.0      19.0
       LAPA                          67.0      35.0
       PINHEIROS                     38.0      13.0
SUL    CAMPO LIMPO                   23.0       7.0
       CAPELA DO SOCORRO              8.0       7.0
       CIDADE ADEMAR                  3.0       4.0
       IPIRANGA                      21.0       5.0
       JABAQUARA                     37.0       5.0
       M BOI MIRIM                   14.0       6.0
       PARELHEIROS                   11.0       7.0
       SANTO AMARO                  101.0       8.0
       VILA MARIANA                  38.0      12.0

In [34]:
df_linhas.to_csv("df_linhas.csv", sep=";", encoding="latin-1", index=False)

In [36]:
df_linha_mae.to_csv("df_linha_mae.csv", sep=";", encoding="latin-1", index=False)

### 3. Tratamento dos Dados

In [20]:
Pre_Process = PreProcessamento()

In [21]:
#Filtro Apenas as linhas que serão utilizadas
df_final = df_final[df_final['Linha'].isin(np.unique(df_linhas[df_linhas['Mantido']=='Mantido']['Linha']))]
df_final.reset_index(drop=True, inplace=True)

In [22]:
df_final[['Ano', 'Volume_Passageiros_BU_VT_DIN']].groupby('Ano').sum('Volume_Passageiros_BU_VT_DIN')/1000

,Volume_Passageiros_BU_VT_DIN
Ano,
2014,1311887.025
2015,1235573.257
2016,1162858.735
2017,1131464.722
2018,1067902.320
2019,546525.575
2020,115872.652
2021,128848.486
2022,363930.283


In [23]:
df_final = Pre_Process.tratar_dados_ausentes_(df_final, 
                                              coluna="Volume_Passageiros_BU_VT_DIN", 
                                              grupo="Linha",
                                              metodo="média")
df_final.reset_index(drop=True, inplace=True)

In [24]:
df_final[['Ano', 'Volume_Passageiros_BU_VT_DIN']].groupby('Ano').sum('Volume_Passageiros_BU_VT_DIN')/10000

,Volume_Passageiros_BU_VT_DIN
Ano,
2014,131196.089108
2015,123563.428604
2016,116288.796248
2017,113171.476710
2018,106888.815672
2019,54806.645221
2020,11825.315616
2021,13116.170968
2022,36531.140888


In [25]:
df_final = Pre_Process.corrigir_outliers(df_final,
                                         coluna="Volume_Passageiros_BU_VT_DIN",
                                         metodo="interquartil")

In [26]:
df_final[['Ano', 'Volume_Passageiros_BU_VT_DIN']].groupby('Ano').sum('Volume_Passageiros_BU_VT_DIN')/10000

,Volume_Passageiros_BU_VT_DIN
Ano,
2014,131182.807108
2015,123560.463804
2016,116288.288448
2017,113171.476710
2018,106888.644172
2019,54806.645221
2020,11825.315616
2021,13116.170968
2022,36530.443588


In [27]:
df_final = Pre_Process.transformacao_dados_(df_final,
                                            coluna="Volume_Passageiros_BU_VT_DIN",
                                            grupo="LINHA_TRATADA", 
                                            metodo="nenhum")
df_final.reset_index(drop=True, inplace=True)

O dataframe não foi alterado


In [28]:
df_final_agrupado_anomes = df_final[['Zona', 'Sub_Prefeitura', 'Linha_Completa', 'Linha', 'Empresa',  'Tipo',  'Nome_Ponto_Inicial', 'Volume_Passageiros_BU_VT_DIN', 'Ano', 'Mes']].groupby(['Ano', 'Mes', 'Zona', 'Sub_Prefeitura', 'Linha_Completa', 'Linha', 'Empresa',  'Tipo',  'Nome_Ponto_Inicial'], as_index=False).sum('Volume_Passageiros_BU_VT_DIN')
df_final_agrupado_anomes.head()

,Ano,Mes,Zona,Sub_Prefeitura,Linha_Completa,Linha,Empresa,Tipo,Nome_Ponto_Inicial,Volume_Passageiros_BU_VT_DIN
0,2014,1,CENTRO,SE,115610 - VILA SABRINA/PC DO CORREIO,115610,SAMBAIBA,CONCESSAO,"PÇA. CARLOS KOSERITZ , 0",136186.0
1,2014,1,CENTRO,SE,115610 - VILA SABRINA/PC DO CORREIO,115610,SAMBAIBA,CONCESSAO,"PÇA. DO CORREIO , 0",136186.0
2,2014,1,CENTRO,SE,117710 - TERM.A.E.CARVALHO-EST.LUZ,117710,VIP II,CONCESSAO,"R. MAUÁ , 790",83168.0
3,2014,1,CENTRO,SE,117810 - SAO MIGUEL-PCA DO CORREIO,117810,VIP II,CONCESSAO,"PÇA. ANTÔNIO ASSIS PEREIRA , 1040",198328.0
4,2014,1,CENTRO,SE,117810 - SAO MIGUEL-PCA DO CORREIO,117810,VIP II,CONCESSAO,"R. RISKALLAH JORGE , 80",198328.0


In [29]:
df_final_agrupado_anomes['Linha_Mae'] = np.where(df_final_agrupado_anomes['Linha'].isin(df_linha_mae['Linha']), "Sim", "Não")

In [30]:
Manipulacao.salvar_arquivo(local_destino, df_final_agrupado_anomes, 'df_final_agrupado_ano_mes')

In [31]:
Manipulacao.salvar_arquivo(local_destino, df_final, 'df_final_tratado')

In [32]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3438312 entries, 0 to 3438311
Data columns (total 26 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   Nome_Arquivo                   object        
 1   Data                           datetime64[ns]
 2   Area                           object        
 3   Empresa                        object        
 4   Linha_Completa                 object        
 5   Tipo                           object        
 6   Total                          float64       
 7   Linha                          object        
 8   Nome_Ponto_Inicial             object        
 9   Sub_Prefeitura                 object        
 10  Lat_Long_Ponto_Inicial         object        
 11  Nome_Ponto_Final               object        
 12  Lat_Long_Ponto_Final           object        
 13  Ano                            int32         
 14  Mes                            int32         
 15  Dia            